In [4]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [12]:
import psycopg2

In [96]:
import numpy as np
import re

In [26]:

conn = psycopg2.connect("dbname=top100 user=postgres password=im4uema3cohb")

In [28]:
cur = conn.cursor()
command = "DROP TABLE IF EXISTS music;"
cur.execute(command)
command = "DROP TABLE IF EXISTS artist;"
cur.execute(command)
cur.close()

In [29]:
commands = (
        """
CREATE TABLE artist(
   id_artist INT GENERATED ALWAYS AS IDENTITY,
   nameArtist VARCHAR(100),
   birth DATE,
   city VARCHAR(100),
   country VARCHAR(100),
   PRIMARY KEY(id_artist)
)
        """,
        """
CREATE TABLE music(
   id_music INT GENERATED ALWAYS AS IDENTITY,
   id_artist INT,
   title VARCHAR(100),
   lyrics VARCHAR(1000),
   genre VARCHAR(100),
   releaseDate DATE,
   ranking INT,
   PRIMARY KEY(id_music),
   CONSTRAINT fk_artist
      FOREIGN KEY(id_artist) 
	  REFERENCES artist(id_artist)
	  ON DELETE CASCADE
)
        """)


In [30]:
cur = conn.cursor()
# create table one by one
for command in commands:
        cur.execute(command)
# close communication with the PostgreSQL database server
cur.close()
# commit the changes
conn.commit()

In [ ]:
sql2 = '''COPY details(employee_id,employee_name,\
employee_email,employee_salary)
FROM '/private/tmp/details.csv'
DELIMITER ','
CSV HEADER;'''
  
cursor.execute(sql2)

In [25]:
if conn is not None:
    conn.close()

In [7]:
import os
import pandas as pd

In [175]:
files = os.listdir("../chart")

dfs_chart = {}
for f in files:
    year = int(f.split("_")[0])
    month = f.split("_")[1].split(".")[0]

    if dfs_chart.get(year) is None:
        dfs_chart[year] = {}
        
    dfs_chart[year][month] = pd.read_csv("../chart/" + f)

In [60]:
dfs_chart[2004]["Juillet"]

,Rank,Artist,Music,Naissance,Pays d'origine,Origine,Nationalité,Pays
0,1,O-zone,Dragostea Din Tei,,Moldavie,,,Moldavie
1,2,K'maro,Femme Like U,,Canada,,,Canada
2,3,Calogero,Face À La Mer,"30 juillet 1971 (51 ans)Échirolles, France",,,Française,
3,4,T-rio,Mamae Eu Quero (choopeta),,,,,
4,5,Le 6-9,Le Poulailler,,,,,"France, Suisse"
...,...,...,...,...,...,...,...,...
95,96,Sandra Lou,Le Banana Split,,,,Française,
96,97,Mickey 3d,Johnny Rep,,France,,,France
97,98,Jean-luc Lahaye,On N'se Reverra Plus,"23 décembre 1952 (69 ans)Paris, France",,,,
98,99,Lmc,Take Me To The Clouds Above,,,,,


In [59]:
dfs_chart[2020]["Fevrier"]["Rank"][0]

1

In [77]:
months = {
        'janvier': 1,
        'février': 2,
        'mars': 3,
        'avril':4,
        'mai':5,
        'juin':6,
        'juillet':7,
        'août':8,
        'septembre':9,
        'octobre':10,
        'novembre':11,
        'décembre':12
       }

In [118]:
def insertNaissance(row):
    texte = re.sub("[\(\[].*?[\)\]]", "", row['Naissance'])
    texte = texte.split()
    #print(texte)
    res = "0000-01-01"
    if len(texte)>2 and texte[0].isnumeric() and texte[2].isnumeric():
        res = str(texte[2])+"-"+str(months[texte[1]])+"-"+str(texte[0])
    return res

In [172]:
def pays(row):
    if not row["Pays"]:
        print(1)
        return row["Pays"]
    elif not row["Nationalité"]:
        print(2)
        return row["Nationalité"]
    elif not row["Origine"]:
        print(3)
        return row["Origine"]
    elif not row["Pays d'origine"]:
        print(4)
        return row["Pays d'origine"]
    else:
        texte = re.sub("[\(\[].*?[\)\]]", "", row['Naissance'])
        texte = texte.split()
        print(texte)
        if len(texte)>3:
            return texte[-1]
        else:
            return ""

In [173]:
for years in dfs_chart:
     for table in dfs_chart[years].values():
          to_insert = table.drop(['Rank', 'Music'], axis=1)
          to_insert['DateNaissance'] = to_insert.apply(lambda row : insertNaissance(row), axis=1)
          to_insert['Pays_to_insert'] = to_insert.apply(lambda row : pays(row), axis=1)

[]
[]
[]
[]
[]
['30', 'juillet', '1971', 'Échirolles,', 'France']
['10', 'février', '1973', "Val-d'Oise,", 'France']
['19', 'juin', '1973', 'Tours,', 'Indre-et-Loire,', 'France']
['4', 'février', '1985', 'Le', 'Mans,', 'Sarthe']
[]
['8', 'janvier', '1965', 'Bergerac']
[]
[]
['19', 'septembre', '1984', 'Staten', 'Island']
[]
[]
['2', 'mai', '1982', 'Le', 'Plessis-Bouchard,', "Val-d'Oise,", 'France']
[]
['3', 'mars', '1974', 'Nigeria']
[]
[]
[]
[]
['9', 'juin', '1983', 'Staten', 'Island']
[]
[]
[]
[]
['13', 'mars', '1992', 'San', 'Juan,', 'Porto', 'Rico,', 'États-Unis']
[]
['17', 'septembre', '1968', 'Chicago,', 'Illinois', 'États-Unis']
['14', 'octobre', '1978', 'Dallas,', 'Texas,', 'États-Unis']
['30', 'septembre', '1977', 'Sainte-Anne,', 'Guadeloupe']
[]
['11', 'janvier', '1981', 'Handsworth,', 'Birmingham,', 'Midlands', 'de', "l'Ouest,", 'Angleterre']
[]
['14', 'septembre', '1973', 'Brooklyn,', 'New', 'York,', 'États-Unis']
['24', 'mars', '1977', 'Fribourg-en-Brisgau']
[]
[]
[]
['24'

In [174]:
to_insert

,Artist,Naissance,Pays d'origine,Origine,Nationalité,Pays,DateNaissance,Pays_to_insert
0,Gazo,5 août 1994 (28 ans)Châteauroux,,,Française,,1994-8-5,Châteauroux
1,Lomepal,"22 novembre 1988 (34 ans)Barcelone, Catalogne(...",,,,,1988-11-22,Catalogne
2,Lomepal,"22 novembre 1988 (34 ans)Barcelone, Catalogne(...",,,,,1988-11-22,Catalogne
3,Soolking,10 décembre 1989 (32 ans)Staouali (d),,,Algérienne,,1989-12-10,Staouali
4,Rema,"1er mai 2000 (22 ans)Benin City, État d’Edo, ...",,,,,0000-01-01,Nigeria
...,...,...,...,...,...,...,...,...
95,Sdm,"28 novembre 1995 (26 ans)Meudon, France",,,,,1995-11-28,France
96,Lizzo,"27 avril 1988 (34 ans)Détroit, Michigan, État...",,,,,1988-4-27,États-Unis
97,Djadja et Dinaz,,France,,,France,0000-01-01,
98,Ckay,27 novembre 1997[réf. nécessaire],,,,,1997-11-27,


In [ ]:
sql = "INSERT INTO artist(nameArtist, birth, city, country) VALUES(%s)"

cur.execute(sql, ("",))